In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import numpy as np
import time


import tkinter as tk
from tkinter import ttk
from PIL import ImageTk, Image
from urllib.request import urlopen
from io import BytesIO
from tkinter import messagebox
import matplotlib.figure
import matplotlib.patches
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import webbrowser
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from matplotlib.colors import ListedColormap
%matplotlib inline


START_WINDOW = tk.Tk()

START_WINDOW.title('THE PREDICTION')

START_WINDOW.geometry('500x200')
START_WINDOW.maxsize(500,200)
START_WINDOW.minsize(500,200)


label = tk.Label(
    text="Select the amount of years for crawling",
    font=(None, 15),
    fg="black",

)
label_window = tk.Label(
    text="Wait until the window will closed and app will start",
    font=(None, 15),
    fg="red",
)

def increase():
    value = int(lbl_value["text"])
    if(value < 10):
        lbl_value["text"] = f"{value + 1}"

    
def decrease():
    value = int(lbl_value["text"])
    if(value > 1): 
        lbl_value["text"] = f"{value - 1}"
    
label.place(relx = 0.5, rely=0.12,anchor = 'center')
label_window.place(relx = 0.5, rely=0.88,anchor = 'center')

START_WINDOW.rowconfigure(0, minsize=50, weight=1)
START_WINDOW.columnconfigure([0, 1, 2], minsize=50, weight=1)

btn_decrease = tk.Button(master=START_WINDOW, text="-",command=decrease)
btn_decrease.grid(row=0, column=0, )

lbl_value = tk.Label(master=START_WINDOW, text="1")
lbl_value.grid(row=0, column=1)

btn_increase = tk.Button(master=START_WINDOW, text="+",command=increase)
btn_increase.grid(row=0, column=2,)

def StopWin():
    START_WINDOW.destroy()
    
def GoCrawling():    
    START_WINDOW.quit()
    
    
    
button = tk.Button(
    text="Start crawling",
    width=35,
    height=1,
    bg="#E1E1E1",
    fg="black",
    command = GoCrawling   
)   

button.place(relx = 0.5, rely=0.73,anchor = 'center')      
START_WINDOW.mainloop()

################################################Crawling section################################################################
################################################################################################################################        

url = 'https://www.soccerbase.com/teams/home.sd'
r = requests.get(url)
soup = bs(r.content, 'html.parser')
teams = soup.find('div', {'class': 'headlineBlock'}, text='Team').next_sibling.find_all('li')
teams_dict = {}
for team in teams:
    link = 'https://www.soccerbase.com' + team.find('a')['href']
    team = team.text
    teams_dict[team] = link
    

team = []
dates = []
comps = []
h_teams = []
a_teams = []
h_scores = []
a_scores = []
teams_list = []
team_pics = [] 


for k, v in teams_dict.items():
    #print('Acquiring %s data...' % k)
    teams_list.append(k)
    pic_flag=0
    headers = ['Team','Competition', 'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Date Keep']
    lst = [0,1, 2, 3, 4, 8, 9, 10,11,12]
    lst = lst[:int(lbl_value["text"])]
    for x in lst:
        y = 153-x 
        r = requests.get('%s&teamTabs=results&season_id=%d' % (v , y))
        soup = bs(r.content, 'html.parser')
        if(pic_flag == 0):
            pic = soup.find('img' , class_="teamFlag iepng")['src']
            team_pics.append(pic)
            pic_flag=1
        h_scores.extend([int(i.text) for i in soup.select('.score a em:first-child')])
        limit_scores = [int(i.text) for i in soup.select('.score a em + em')]
        a_scores.extend([int(i.text) for i in soup.select('.score a em + em')])
        
        limit = len(limit_scores)
        team.extend([k for i in soup.select('.tournament', limit=limit)])
        comps.extend([i.text for i in soup.select('.tournament a', limit=limit)])
        dates.extend([i.text for i in soup.select('.dateTime .hide', limit=limit)])
        h_teams.extend([i.text for i in soup.select('.homeTeam a', limit=limit)])
        a_teams.extend([i.text for i in soup.select('.awayTeam a', limit=limit)])
        
################################################################################################################################        
################################################################################################################################        


#############################################DataFrames section#################################################################        
################################################################################################################################        

df = pd.DataFrame(list(zip(team,comps, h_teams, h_scores, a_teams, a_scores, dates)) ,columns=headers)

df_summary = pd.DataFrame(index=teams_list,columns=['Games played'])

df_summary['Games played'] = 0
df_summary['Wins at home'] = 0
df_summary['Draws at home'] = 0
df_summary['Loses at home'] = 0
df_summary['Goals at home'] = 0
df_summary['Conceded goals at home'] = 0
df_summary['Wins at away'] = 0
df_summary['Draws at away'] = 0
df_summary['Loses at away'] = 0
df_summary['Goals at away'] = 0
df_summary['Conceded goals at away'] = 0

for Team,HomeTeam,HomeScore,AwayScore in zip(team, h_teams, h_scores, a_scores):
    
    if(Team == HomeTeam): #TEAM play at home   
        df_summary.loc[[Team], ['Games played']] += 1
        df_summary.loc[[Team], ['Goals at home']] += HomeScore #TEAM score goals at home
        df_summary.loc[[Team], ['Conceded goals at home']] += AwayScore #TEAM score goals at home
        if(HomeScore > AwayScore):
            df_summary.loc[[Team], ['Wins at home']] += 1 #TEAM win at home
        elif(HomeScore == AwayScore):
            df_summary.loc[[Team], ['Draws at home']] += 1 #TEAM draw at home
        else:
            df_summary.loc[[Team], ['Loses at home']] += 1 #TEAM lose at home
    else: #Team play away
        df_summary.loc[[Team], ['Games played']] += 1
        df_summary.loc[[Team], ['Goals at away']] += AwayScore #TEAM score goals at home
        df_summary.loc[[Team], ['Conceded goals at away']] += HomeScore #TEAM score goals at home
        if(HomeScore < AwayScore):
            df_summary.loc[[Team], ['Wins at away']] += 1 #TEAM win away
        elif(HomeScore == AwayScore):
            df_summary.loc[[Team], ['Draws at away']] += 1 #TEAM draw away
        else:
            df_summary.loc[[Team], ['Loses at away']] += 1 # Team lose away
            
StopWin()
    
#df.to_excel("AllData.xlsx") #Export all crawling data to exle 
#df_summary.to_excel("SummaryOfData.xlsx") #Export Summary data to exle 
#df                  #Crawling data
#df_summary          #Summary data

###################################################################################################################################
###################################################################################################################################


#########################################Application section####################################################################        
################################################################################################################################        


window = tk.Tk()

window.title('THE PREDICTION')
window.geometry('1450x700')
window.maxsize(1450,700)
window.minsize(1450,700)

#__________ Labels_______#
Title = tk.Label(
    text="Is it possible to predict the results of home games ?",
    fg="black",
    font=(None, 28)
)
Team_label = tk.Label(
    text="Select Team",
    font=(None, 15),
    fg="black",
)

Team_label.place(relx = 0.10, rely=0.12,anchor = 'center')
Title.place(relx = 0.5, rely=0.07,anchor = 'center')




#__________________Frames_________________#
Team_Frame = tk.Frame(master=window, width=1440, height=240, bg="#E1E1E1")
Team_Frame.place(relx = 0.5, rely=0.332,anchor = 'center')

Predict_Frame = tk.Frame(master=window, width=1440, height=300, bg="#E1E1E1")
Predict_Frame.place(relx = 0.5, rely=0.78,anchor = 'center')

Pic_Frame = tk.Frame(master=window, width=186, height=186)
Pic_Frame.place(relx = 0.1, rely=0.35,anchor = 'center')


#________________Drop downs selected and activation__________________#
def set_data(team_name):
    for Team in zip(teams_list):
        if(Team[0] == team_name):
            TotalGamesAtHome = (df_summary.loc[team_name]['Wins at home'])+(df_summary.loc[team_name]['Draws at home'])+(df_summary.loc[team_name]['Loses at home'])
            TotalGamesAway = (df_summary.loc[team_name]['Wins at away'])+(df_summary.loc[team_name]['Draws at away'])+(df_summary.loc[team_name]['Loses at away'])
        
        
            Total_details = tk.Text(window, height=13, width=47)
            Total_details.place(relx = 0.33, rely=0.33,anchor = 'center')
            
            Home_details = tk.Text(window, height=13, width=47)
            Home_details.place(relx = 0.595, rely=0.33,anchor = 'center')
            
            Away_details = tk.Text(window, height=13, width=47)
            Away_details.place(relx = 0.86, rely=0.33,anchor = 'center')
                       
            
            mylabels = ["Wins", "Draws", "Loses"]  
            
            TextStr = "----Summary---- " 
            Total_details.insert(tk.END, TextStr)
            TextStr = "\nMatches played: " + str((df_summary.loc[team_name]['Games played']))
            Total_details.insert(tk.END, TextStr)
            TextStr = "\nTotal wins: " + str((df_summary.loc[team_name]['Wins at home'])+(df_summary.loc[team_name]['Wins at away']))
            Total_details.insert(tk.END, TextStr)
            TextStr = "\nTotal draws: " + str((df_summary.loc[team_name]['Draws at home'])+(df_summary.loc[team_name]['Draws at away']))
            Total_details.insert(tk.END, TextStr)
            TextStr = "\nTotal loses " + str((df_summary.loc[team_name]['Loses at home'])+(df_summary.loc[team_name]['Loses at away']))
            Total_details.insert(tk.END, TextStr)
            TextStr = "\nTotal goals: " + str((df_summary.loc[team_name]['Goals at home'])+(df_summary.loc[team_name]['Goals at away']))
            Total_details.insert(tk.END, TextStr)
            TextStr = "\nTotal Conceded: " + str((df_summary.loc[team_name]['Conceded goals at home'])+(df_summary.loc[team_name]['Conceded goals at away']))
            Total_details.insert(tk.END, TextStr)
            
            a1=((df_summary.loc[team_name]['Wins at home'])+(df_summary.loc[team_name]['Wins at away']))/df_summary.loc[team_name]['Games played']
            a2=((df_summary.loc[team_name]['Draws at home'])+(df_summary.loc[team_name]['Draws at away']))/df_summary.loc[team_name]['Games played']
            a3=((df_summary.loc[team_name]['Loses at home'])+(df_summary.loc[team_name]['Loses at away']))/df_summary.loc[team_name]['Games played']
            y = np.array([a1, a2, a3])
            fig = matplotlib.figure.Figure(figsize=(2.9,2.9))
            ax = fig.add_subplot(111)
            ax.pie([a1,a2,a3],autopct='%1.1f%%',normalize=False) 
            ax.legend(mylabels, bbox_to_anchor=(1,0), loc="lower right",  bbox_transform=fig.transFigure)
            circle=matplotlib.patches.Circle( (0,0), 0.1, color='white')
            ax.add_artist(circle)
            canvas = FigureCanvasTkAgg(fig, master=window)
            canvas.get_tk_widget().place(relx = 0.388, rely=0.33,anchor = 'center')
            canvas.draw()
                      
            
            TextStr = "------Home------ " 
            Home_details.insert(tk.END, TextStr)
            TextStr = "\nMatches played: " + str(TotalGamesAtHome)
            Home_details.insert(tk.END, TextStr)
            TextStr = "\nWins at home: " + str(df_summary.loc[team_name]['Wins at home'])
            Home_details.insert(tk.END, TextStr)
            TextStr = "\nDraws at home: " + str(df_summary.loc[team_name]['Draws at home'])
            Home_details.insert(tk.END, TextStr)
            TextStr = "\nLoses at home: " + str(df_summary.loc[team_name]['Loses at home'])
            Home_details.insert(tk.END, TextStr)
            TextStr = "\nGoals at home: " + str(df_summary.loc[team_name]['Goals at home'])
            Home_details.insert(tk.END, TextStr)
            TextStr = "\nConceded at home: " + str(df_summary.loc[team_name]['Conceded goals at home'])
            Home_details.insert(tk.END, TextStr)
            
            a1=(df_summary.loc[team_name]['Wins at home'])/TotalGamesAtHome
            a2=(df_summary.loc[team_name]['Draws at home'])/TotalGamesAtHome
            a3=(df_summary.loc[team_name]['Loses at home'])/TotalGamesAtHome
            y = np.array([a1, a2, a3])
            fig = matplotlib.figure.Figure(figsize=(2.9,2.9))
            ax = fig.add_subplot(111)
            ax.pie([a1,a2,a3],autopct='%1.1f%%',normalize=False) 
            ax.legend(mylabels, bbox_to_anchor=(1,0), loc="lower right",  bbox_transform=fig.transFigure)
            circle=matplotlib.patches.Circle( (0,0), 0.1, color='white')
            ax.add_artist(circle)
            canvas = FigureCanvasTkAgg(fig, master=window)
            canvas.get_tk_widget().place(relx = 0.653, rely=0.33,anchor = 'center')
            canvas.draw()                     
                      
            
            TextStr = "------Away------ " 
            Away_details.insert(tk.END, TextStr)
            TextStr = "\nMatches played: " + str(TotalGamesAway)
            Away_details.insert(tk.END, TextStr)
            TextStr = "\nWins at away: " + str(df_summary.loc[team_name]['Wins at away'])
            Away_details.insert(tk.END, TextStr)
            TextStr = "\nDraws at away: " + str(df_summary.loc[team_name]['Draws at away'])
            Away_details.insert(tk.END, TextStr)
            TextStr = "\nLoses at away: " + str(df_summary.loc[team_name]['Loses at away'])
            Away_details.insert(tk.END, TextStr)
            TextStr = "\nGoals at away: " + str(df_summary.loc[team_name]['Goals at away'])
            Away_details.insert(tk.END, TextStr)
            TextStr = "\nConceded at away: " + str(df_summary.loc[team_name]['Conceded goals at away'])
            Away_details.insert(tk.END, TextStr)
            
          
            a1=(df_summary.loc[team_name]['Wins at away'])/TotalGamesAway
            a2=(df_summary.loc[team_name]['Draws at away'])/TotalGamesAway
            a3=(df_summary.loc[team_name]['Loses at away'])/TotalGamesAway
            y = np.array([a1, a2, a3])
            fig = matplotlib.figure.Figure(figsize=(2.9,2.9))
            ax = fig.add_subplot(111)
            ax.pie([a1,a2,a3],autopct='%1.1f%%',normalize=False) 
            ax.legend(mylabels, bbox_to_anchor=(1,0), loc="lower right",  bbox_transform=fig.transFigure)          
            circle=matplotlib.patches.Circle( (0,0), 0.1, color='white')
            ax.add_artist(circle)
            canvas = FigureCanvasTkAgg(fig, master=window)
            canvas.get_tk_widget().place(relx = 0.918, rely=0.33,anchor = 'center')
            canvas.draw()
            
            
            button.place(relx = 0.1, rely=0.535,anchor = 'center')
            
            
#__________________Drop downs_________________#
n = tk.StringVar()
h_team_dropDown = ttk.Combobox(window, width = 35, textvariable = n)
h_team_dropDown['values'] = teams_list
h_team_dropDown.place(relx = 0.10, rely=0.19,anchor = 'center')


def set_home(_):
    global HTEAM
    HTEAM = h_team_dropDown.get()
    set_pic(HTEAM)
    set_data(HTEAM)    
h_team_dropDown.bind("<<ComboboxSelected>>", set_home)    



def set_pic(team_name):
    for x, y  in zip(teams_list, team_pics):
        if(team_name == x):
            URL = y
            u = urlopen(URL)
            raw_data = u.read()
            u.close()
            im = Image.open(BytesIO(raw_data))
            im2 = im.resize((180,180))
            photo = ImageTk.PhotoImage(im2)
            label = tk.Label(image=photo)
            label.image = photo 
            label.place(relx = 0.10, rely=0.35,anchor = 'center')    
    




def sum_games(firstTeam,secondTeam):  #first - home team second - away team
    Total_games = 0
    for HomeTeam,AwayTeam,HomeScore,AwayScore in zip(h_teams,a_teams, h_scores, a_scores):
        if(HomeTeam == firstTeam):
            if(AwayTeam == secondTeam):
                Total_games += 1
                
    return Total_games


#__________________________Clculation button____________________#

def calculate():
    Prediction_title = tk.Label(
    text="Successful predictions",
    fg="black",bg="#E1E1E1",
    font=(None, 13))
    Prediction_title.place(relx = 0.10, rely=0.59,anchor = 'center')
    
    ACC_title = tk.Label(
    text="Prediction accuracy",
    fg="black",bg="#E1E1E1",
    font=(None, 13))
    ACC_title.place(relx = 0.30, rely=0.59,anchor = 'center')
    
    partition_title = tk.Label(
    text="Partition of successful prediction",
    fg="black",bg="#E1E1E1",
    font=(None, 13))
    partition_title.place(relx = 0.5, rely=0.59,anchor = 'center')
    
    Pre_summary_title = tk.Label(
    text="Summary of prediction",
    fg="black",bg="#E1E1E1",
    font=(None, 13))
    Pre_summary_title.place(relx = 0.8, rely=0.61,anchor = 'center')    
    
    counter = 0
    Result = []
    HomeH2Havg = []
    AwayH2Havg = []
    HomeGoalsavg = []
    AwayGoalsavg = []
    for Team,HomeTeam,AwayTeam,HomeScore,AwayScore in zip(team,h_teams,a_teams, h_scores, a_scores):
        if(Team == HTEAM):
            if AwayTeam in teams_list:
                if(HTEAM == HomeTeam):
                    T_games = sum_games(HTEAM,AwayTeam)
                    HomeH2Havg.append(HomeScore/T_games)
                    AwayH2Havg.append(AwayScore/T_games)
                    HomeGoalsavg.append((df_summary.loc[HomeTeam]['Goals at home'])/((df_summary.loc[HomeTeam]['Wins at home'])+(df_summary.loc[HomeTeam]['Draws at home'])+(df_summary.loc[HTEAM]['Loses at home'])))
                    AwayGoalsavg.append((df_summary.loc[AwayTeam]['Goals at away'])/((df_summary.loc[AwayTeam]['Wins at away'])+(df_summary.loc[AwayTeam]['Draws at away'])+(df_summary.loc[AwayTeam]['Loses at away'])))
                    if(HomeScore>AwayScore):
                        Result.append(1) # home team win
                    elif(HomeScore<AwayScore):
                        Result.append(2) # home team lose
                    else:
                        Result.append(3) # draw
    
    
    df_X = pd.DataFrame(list(zip(HomeH2Havg,AwayH2Havg,HomeGoalsavg,AwayGoalsavg,Result)) ,columns=['HomeH2Havg','AwayH2Havg','HomeGoalsavg','AwayGoalsavg','Result'])
     
        
    XXX=['HomeH2Havg','AwayH2Havg','HomeGoalsavg','AwayGoalsavg']
    X = df_X[XXX].values
    y = df_X["Result"].values    
    XTrain, XTest, yTrain, yTest = train_test_split(X, y, random_state=1, test_size=0.2)
    gnb = GaussianNB()  
    gnb.fit(XTrain,yTrain)
    y_pred = gnb.predict(XTest)
    y_pred_train = gnb.predict(XTrain)     
    
    
    resDF=pd.DataFrame({"Actual":yTest,"Predicted":y_pred})
    resDF['correct'] = 0
    for i in range(0,len(yTest)):
        if(resDF.loc[i]['Actual'] == resDF.loc[i]['Predicted']):
            resDF.at[i,"correct"] = 1
        else:
            resDF.at[i,"correct"] = 0            
    #resDF.to_excel("Prediction.xlsx")  #Export prediction table to exle        
    win_count = 0
    draw_count = 0
    lose_count = 0
    right_predicts = 0
    for i in range(0,len(yTest)):
        if(resDF.loc[i]["correct"] == 1):
            right_predicts +=1
            if(resDF.loc[i]["Predicted"] == 1):
                win_count += 1
            elif(resDF.loc[i]["Predicted"] == 2):
                lose_count += 1
            else:
                draw_count +=1
          
        
    mylabels = ["Right Prediction", "Wrong Prediction"] 
    a1=right_predicts/len(yTest)
    a2=(len(yTest)-right_predicts)/len(yTest)
    y = np.array([a1, a2])
    fig = matplotlib.figure.Figure(figsize=(3.5,3.5))
    ax = fig.add_subplot(111)
    ax.pie([a1,a2],autopct='%1.1f%%',normalize=False) 
    ax.legend(mylabels, bbox_to_anchor=(1,0), loc="lower right",  bbox_transform=fig.transFigure)     
    circle=matplotlib.patches.Circle( (0,0), 0.1, color='white')
    ax.add_artist(circle)
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.get_tk_widget().place(relx = 0.10, rely=0.8,anchor = 'center')
    canvas.draw()          
    
    
    data1 = {'X': ['Test','Train'],
         'Percent': [metrics.accuracy_score(y_true = yTest, y_pred = y_pred)
                     ,metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train)]}
    df1 = pd.DataFrame(data1,columns=['X','Percent'])
    figure1 = plt.Figure(figsize=(3.5,3.5))
    ax1 = figure1.add_subplot(111)
    bar1 = FigureCanvasTkAgg(figure1, window)
    bar1.get_tk_widget().place(relx = 0.3, rely=0.8,anchor = 'center')
    df1 = df1[['X','Percent']].groupby('X').sum()
    df1.plot(kind='bar', legend=True, ax=ax1)
  

                  
    mylabels = ["Wins", "Draws", "Loses"] 
    a1=win_count/right_predicts
    a2=draw_count/right_predicts
    a3=lose_count/right_predicts
    y = np.array([a1, a2, a3])
    fig = matplotlib.figure.Figure(figsize=(3.5,3.5))
    ax = fig.add_subplot(111)
    ax.pie([a1,a2,a3],autopct='%1.1f%%',normalize=False) 
    ax.legend(mylabels, bbox_to_anchor=(1,0), loc="lower right",  bbox_transform=fig.transFigure)
            
    circle=matplotlib.patches.Circle( (0,0), 0.1, color='white')
    ax.add_artist(circle)
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.get_tk_widget().place(relx = 0.5, rely=0.8,anchor = 'center')
    canvas.draw()    
    
    
    
    Summary_details = tk.Text(window, height=13, width=54)
    Summary_details.place(relx = 0.8, rely=0.8,anchor = 'center')
    
    TotalGamesAtHome = (df_summary.loc[HTEAM]['Wins at home'])+(df_summary.loc[HTEAM]['Draws at home'])+(df_summary.loc[HTEAM]['Loses at home'])
    TotalGamesAway = (df_summary.loc[HTEAM]['Wins at away'])+(df_summary.loc[HTEAM]['Draws at away'])+(df_summary.loc[HTEAM]['Loses at away'])

    
    a1=(right_predicts/len(yTest))*100
    
    TextStr = "-----Home advantage?----- " 
    Summary_details.insert(tk.END, TextStr)
  
    if((df_summary.loc[HTEAM]['Wins at home'])/TotalGamesAtHome > (df_summary.loc[HTEAM]['Wins at away'])/TotalGamesAway):
        TextStr = "\nFrom the data it can be seen that " + HTEAM + " win \nat higher precentage in the home games, than in the \naway games"
        Summary_details.insert(tk.END, TextStr)
    else:
        TextStr = "\nIt is wrong to say that " + HTEAM + " has a home\nadvantage over the away games"
        Summary_details.insert(tk.END, TextStr)
        
    TextStr = "\n\n----Prediction summary---- " 
    Summary_details.insert(tk.END, TextStr)
    
    if(a1 == 100):
        TextStr = "\nIt can be said that the algorithm predicts 100%\n" +HTEAM + " games" 
        Summary_details.insert(tk.END, TextStr)
    elif(a1 >= 80):
        TextStr = "\nIt can be said that the algorithm well predicts\n" +HTEAM + " games" 
        Summary_details.insert(tk.END, TextStr)
    elif(a1 > 60):
        TextStr = "\nIt can be said that the algorithm moderately predicts\n" +HTEAM + " games" 
        Summary_details.insert(tk.END, TextStr)
    else:
        TextStr = "\nIt can be said that the algorithm badly predicts\n" +HTEAM + " games" 
        Summary_details.insert(tk.END, TextStr)   
    
    a1=(win_count/right_predicts)*100
    a2=(draw_count/right_predicts)*100
    a3=(lose_count/right_predicts)*100
    if(a1 > a2):
        if(a1 > a3):
            TextStr = "\nThe algorithm best predicts the victories" 
            Summary_details.insert(tk.END, TextStr)
    if(a2 > a1):
        if(a2 > a3):
            TextStr = "\nThe algorithm best predicts the draws" 
            Summary_details.insert(tk.END, TextStr)
    if(a3 > a1):
        if(a3 > a2):
            TextStr = "\nThe algorithm best predicts the losses" 
            Summary_details.insert(tk.END, TextStr)  
    
    
button = tk.Button(
    text="Predict",
    width=35,
    height=1,
    bg="#E1E1E1",
    fg="black",
    command = calculate 
)



window.mainloop()

#####################################################END########################################################################        
################################################################################################################################        
